In [1]:
import gc
import pandas as pd
import numpy as np
import os
import json
import sklearn.metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from scipy.sparse import dok_matrix, coo_matrix
from sklearn.utils.multiclass import  type_of_target
path = "data"

## 데이터 로딩

In [2]:
%%time
aisles = pd.read_csv(os.path.join(path, "aisles.csv"), dtype={'aisle_id': np.uint8, 'aisle': 'category'})
departments = pd.read_csv(os.path.join(path, "departments.csv"),
                          dtype={'department_id': np.uint8, 'department': 'category'})
order_prior = pd.read_csv(os.path.join(path, "order_products__prior.csv"), dtype={'order_id': np.uint32,
                                                                                  'product_id': np.uint16,
                                                                                  'add_to_cart_order': np.uint8,
                                                                                  'reordered': bool})
order_train = pd.read_csv(os.path.join(path, "order_products__train.csv"), dtype={'order_id': np.uint32,
                                                                                  'product_id': np.uint16,
                                                                                  'add_to_cart_order': np.uint8,
                                                                                  'reordered': bool})
orders = pd.read_csv(os.path.join(path, "orders.csv"), dtype={'order_id': np.uint32,
                                                              'user_id': np.uint32,
                                                              'eval_set': 'category',
                                                              'order_number': np.uint8,
                                                              'order_dow': np.uint8,
                                                              'order_hour_of_day': np.uint8
                                                              })

products = pd.read_csv(os.path.join(path, "products.csv"), dtype={'product_id': np.uint16,
                                                                  'aisle_id': np.uint8,
                                                                  'department_id': np.uint8})

## preprocessing#2 의 labels : 주문과 상품이 reordered 된 것인가?
labels = pd.read_pickle(os.path.join(path, 'chunk_0.pkl'))
## preprocessing#1 의 labels : 어떤 유저가 어떤 상품을 샀는가?
user_product = pd.read_pickle(os.path.join(path, 'previous_products.pkl'))

CPU times: user 10.3 s, sys: 1.13 s, total: 11.4 s
Wall time: 11.4 s


## 전처리 #3

In [32]:
orders.head(12)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [34]:
%%time
## orders 데이터에서 'user_id', 'order_number' 로 group_by한다.
gb = orders[['user_id', 'order_number', 'days_since_prior_order']].groupby(['user_id', 'order_number'])

CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 29.6 ms


In [35]:
## gb에서 days_since_prior_order 열 (과거 주문으로 부터 얼마나(일) 지났는가?) 만 갖고와 정리한다.
## 즉 아래 처럼 유저 1은 order_number 2 까지 15일 3까지 21일 걸렸다는 정보를 order_comsum에 정리해 저장함
## 그러면 왜 group_by를 2-level을 줬을까? -->> order_number 대로 빠르고 간편하게 sorting하기 위함

order_comsum = gb['days_since_prior_order'].sum()

In [36]:
gb.get_group((1,1))

,user_id,order_number,days_since_prior_order
0,1,1,NaN


In [37]:
gb.get_group((1,2))

,user_id,order_number,days_since_prior_order
1,1,2,15.0


In [38]:
gb.get_group((1,3))

,user_id,order_number,days_since_prior_order
2,1,3,21.0


In [39]:
order_comsum.head(12)

user_id  order_number
1        1                NaN
         2               15.0
         3               21.0
         4               29.0
         5               28.0
         6               19.0
         7               20.0
         8               14.0
         9                0.0
         10              30.0
         11              14.0
2        1                NaN
Name: days_since_prior_order, dtype: float64

In [40]:
%%time
## 결국 다시 1-level(user_id)로 groupby해서 cumsum(누적합) 한다.
## 그 결과 처음 부터 마지막 order 까지 얼마나 기간이 소요됐는지가 나온다. 
## 또한 이 사람이 얼마나 오래된 사람인지도 구분 할 수 있게 된다.

order_comsum = order_comsum.groupby(level=[0]).cumsum()

CPU times: user 104 ms, sys: 48 ms, total: 152 ms
Wall time: 152 ms


In [44]:
order_comsum.head(12)

user_id  order_number
1        1                 NaN
         2                15.0
         3                36.0
         4                65.0
         5                93.0
         6               112.0
         7               132.0
         8               146.0
         9               146.0
         10              176.0
         11              190.0
2        1                 NaN
Name: days_since_prior_order, dtype: float64

## 파생변수 #1 : days_since_prior_order_comsum 첫 구매로 부터 마지막 오더까지 얼마나 걸렸나.

In [46]:
## 위 처럼 인덱스가 꼬여 있으니 reset_index로 정리하고
## days_since_prior_order 칼럼을 days_since_prior_order_comsum로 변경한다.-->파생변수 하나 만들어짐
order_comsum = order_comsum.reset_index().rename(columns={'days_since_prior_order':'days_since_prior_order_comsum'})

In [48]:
order_comsum.head()

,user_id,order_number,days_since_prior_order_comsum
0,1,1,NaN
1,1,2,15.0
2,1,3,36.0
3,1,4,65.0
4,1,5,93.0


In [49]:
orders.head(1)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN


In [50]:
%%time
## 새로 만든 파생변수 days_since_prior_order_comsum 를 order에 추가한다.
order_comsum = pd.merge(order_comsum, orders, on=['user_id', 'order_number'])

CPU times: user 1.5 s, sys: 168 ms, total: 1.67 s
Wall time: 1.67 s


In [51]:
## 필요한 칼럼만 빼온다.
order_comsum = order_comsum[['user_id', 'order_number', 'days_since_prior_order_comsum', 'order_id']]

In [52]:
order_comsum.head()

,user_id,order_number,days_since_prior_order_comsum,order_id
0,1,1,NaN,2539329
1,1,2,15.0,2398795
2,1,3,36.0,473747
3,1,4,65.0,2254736
4,1,5,93.0,431534


In [54]:
order_prior.head(1)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,True


In [55]:
%%time
order_product = pd.merge(order_prior, orders, on='order_id')[['order_id', 'product_id', 'eval_set']]

CPU times: user 6.25 s, sys: 2.62 s, total: 8.87 s
Wall time: 8.87 s


In [57]:
## prior 의 오더-제품 eval정보
order_product.head(1)

,order_id,product_id,eval_set
0,2,33120,prior


In [58]:
## train_test 의 오더-제품 eval 정보
order_product_train_test = labels[['order_id', 'product_id', 'eval_set']]

In [59]:
order_product_train_test.head()

,order_id,product_id,eval_set
0,1,49302,train
1,1,11109,train
2,1,43633,train
3,1,22035,train
4,36,19660,train


In [60]:
## 그냥 concat함
## 이유 order_comsum과 합치려고
order_product = pd.concat([order_product, order_product_train_test])

In [62]:
order_product.head(1)

,order_id,product_id,eval_set
0,2,33120,prior


In [63]:
## order_comsum를 합쳐 파생변수를 추가한다.
order_product = pd.merge(order_product, order_comsum, on='order_id')

In [65]:
order_product.head(1)

,order_id,product_id,eval_set,user_id,order_number,days_since_prior_order_comsum
0,2,33120,prior,202279,3,28.0


In [69]:
user_product.head(1)

,user_id,product_id
0,202279,33120


In [66]:
%%time
order_product = pd.merge(order_product, user_product, on=['user_id', 'product_id'])

CPU times: user 18.6 s, sys: 5.47 s, total: 24 s
Wall time: 24 s


In [67]:
order_product.head(2)

,order_id,product_id,eval_set,user_id,order_number,days_since_prior_order_comsum
0,2,33120,prior,202279,3,28.0
1,104690,33120,prior,202279,8,151.0


In [70]:
%%time
## 유저가 같은 상품을 구매하는데 얼마나 걸리는지 계산하는것(np.diff)
## 196번 상품의 경우 유저 1은 15일 21일 .....이렇게 구매했고
## 만약 재 구매하지 않았다면 NaN이 돼기 때문에 0.0으로 NaN을 표시함(nan_to_num)
temp = order_product.groupby(['user_id', 'product_id', 'order_number'])['days_since_prior_order_comsum'].sum().groupby(level=[0, 1]).apply(lambda x: np.diff(np.nan_to_num(x)))


CPU times: user 38min 20s, sys: 57.2 s, total: 39min 17s
Wall time: 38min 11s


In [71]:
temp.head()

user_id  product_id
1        196           [15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0...
         10258         [21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0, 30.0...
         10326                                                    [97.0]
         12427         [15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0...
         13032                                       [117.0, 44.0, 14.0]
Name: days_since_prior_order_comsum, dtype: object

In [72]:
#temp_bc = temp.copy()

In [73]:
## group_by 때문에 index level이 2 이다. 따라서 한번에 to_frame으로 
## 데이터 프레임으로 변환
temp = temp.to_frame('periods').reset_index()

In [75]:
temp.head()

,user_id,product_id,periods
0,1,196,"[15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0..."
1,1,10258,"[21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0, 30.0..."
2,1,10326,[97.0]
3,1,12427,"[15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0..."
4,1,13032,"[117.0, 44.0, 14.0]"


In [ ]:
aggregated = temp.copy()

In [77]:
%%time
## 마지막 기간만 갖고온다.
aggregated['last'] = aggregated.periods.apply(lambda x: x[-1])

In [78]:
%%time
## 마지막에서 2번째 전을 갖고온다 다만 periods 가 2개 이상인 경우만 계산하고 만약 아니라면 nan 반환
aggregated['prev1'] = aggregated.periods.apply(lambda x: x[-2] if len(x) > 1 else np.nan)

CPU times: user 6.72 s, sys: 372 ms, total: 7.09 s
Wall time: 7.08 s


In [80]:
## 3번째 97일 걸린 상품 1개 밖에 없어서 NaN반환
aggregated.head()

,user_id,product_id,periods,last,prev1,prev2
0,1,196,"[15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0...",14.0,30.0,0.0
1,1,10258,"[21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0, 30.0...",14.0,30.0,0.0
2,1,10326,[97.0],97.0,NaN,NaN
3,1,12427,"[15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0...",14.0,30.0,0.0
4,1,13032,"[117.0, 44.0, 14.0]",14.0,44.0,117.0


In [79]:
%%time
## 마찬가지 마지막으로 3번째 전
aggregated['prev2'] = aggregated.periods.apply(lambda x: x[-3] if len(x) > 2 else np.nan)

CPU times: user 6.69 s, sys: 244 ms, total: 6.93 s
Wall time: 6.93 s


In [81]:
%%time
## 구매 기간 중앙값
aggregated['median'] = aggregated.periods.apply(lambda x: np.median(x[:-1]))

/home/supermario0759/anaconda3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


CPU times: user 6min 47s, sys: 912 ms, total: 6min 47s
Wall time: 6min 47s


In [82]:
%%time
## 평균
aggregated['mean'] = aggregated.periods.apply(lambda x: np.mean(x[:-1]))

/home/supermario0759/anaconda3/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2909: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


CPU times: user 2min 31s, sys: 612 ms, total: 2min 31s
Wall time: 2min 31s


In [83]:
aggregated.head()

,user_id,product_id,periods,last,prev1,prev2,median,mean
0,1,196,"[15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0...",14.0,30.0,0.0,20.0,19.555556
1,1,10258,"[21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0, 30.0...",14.0,30.0,0.0,20.5,20.125000
2,1,10326,[97.0],97.0,NaN,NaN,NaN,NaN
3,1,12427,"[15.0, 21.0, 29.0, 28.0, 19.0, 20.0, 14.0, 0.0...",14.0,30.0,0.0,20.0,19.555556
4,1,13032,"[117.0, 44.0, 14.0]",14.0,44.0,117.0,80.5,80.500000


In [84]:
aggregated.to_pickle('data/product_periods_stat_origin.pkl')

In [85]:
aggregated.drop('periods', axis=1, inplace=True)

In [86]:
aggregated.head()

,user_id,product_id,last,prev1,prev2,median,mean
0,1,196,14.0,30.0,0.0,20.0,19.555556
1,1,10258,14.0,30.0,0.0,20.5,20.125000
2,1,10326,97.0,NaN,NaN,NaN,NaN
3,1,12427,14.0,30.0,0.0,20.0,19.555556
4,1,13032,14.0,44.0,117.0,80.5,80.500000
